<a href="https://colab.research.google.com/github/brunotagoncalves/publicdataml/blob/main/Estrutura%C3%A7%C3%A3o_de_Dados_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.	Estruturação de Dados
Notebook para Instalação em **Spark**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget http://www-eu.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz


--2021-02-19 01:58:50--  http://www-eu.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving www-eu.apache.org (www-eu.apache.org)... 95.216.26.30, 2a01:4f9:2a:1a61::2
Connecting to www-eu.apache.org (www-eu.apache.org)|95.216.26.30|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz [following]
--2021-02-19 01:58:50--  https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 2a01:4f8:10a:201a::2
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233333392 (223M) [application/x-gzip]
Saving to: ‘spark-2.4.7-bin-hadoop2.7.tgz’

spark-2.4.7-bin-had 100%[===================>] 222.52M  24.1MB/s    in 9.9s    

2021-02-19 01:59:01 (22.6 MB/s) - ‘spark-2.4.7-bin-hadoop2.7.tgz’ saved [23333

In [1]:
!tar -xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

In [9]:
!apt-get update
!apt-get install p7zip-full
!p7zip -d -k drive/MyDrive/publicdataml/*.7z # arquivos do ftp 

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
B

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [155]:
df = spark.read.format('csv').option("sep", ";").option("header", "True").load('*.txt')

In [5]:

df.count()

30189752

In [16]:
test = df.limit(10)
test.toPandas()

,competência,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,idade,horascontratuais,raçacor,sexo,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,fonte
0,202009,3,31,314930,C,2513600,-1,715615,101,5,39,44,3,1,0,1,31,0,0,0,2474.080078125,8,0,1
1,202009,3,31,316470,M,6920601,1,411005,101,7,37,40,1,3,0,1,20,0,0,0,1250,5,0,1
2,202009,5,52,521640,A,116499,-1,641015,101,5,35,44,3,1,2,3,31,0,0,0,1506,99,0,1
3,202009,2,29,290340,B,810002,1,517420,101,1,58,44,6,1,0,1,20,0,0,0,1164,1,0,1
4,202009,3,33,330285,Q,8630503,-1,411005,103,6,17,24,3,1,0,1,43,0,0,0,581.1599731445313,5,1,1
5,202009,2,27,270430,P,8520100,-1,331110,101,7,21,44,6,3,0,1,31,0,0,0,998,5,0,1
6,202009,3,35,354980,G,4711301,1,421125,101,7,20,44,3,1,0,1,20,0,0,0,1561,1,0,1
7,202009,2,29,291920,N,7820500,1,511210,111,7,25,0,3,1,0,1,20,0,1,0,6204,10,0,1
8,202009,2,29,292740,F,4221902,1,731125,101,4,32,44,3,1,0,4,20,0,0,0,2228.60009765625,9,0,1
9,202009,4,42,420910,M,7112000,-1,711320,101,4,26,44,1,1,0,1,40,0,0,0,1500,5,0,1


In [156]:
import pyspark.sql.functions as F
df = df.withColumn('idade',F.col('idade').cast('int'))
df = df.withColumn('horascontratuais',F.col('horascontratuais').cast('int'))
df = df.withColumn('salário',F.col('salário').cast('float'))

In [157]:
df = df.filter("""`salário` BETWEEN 400 AND 100000""")

In [158]:
df = df.withColumn('faixapnad',F.expr("""Case
WHEN `salário` <= 640 THEN '1º Quintil PNAD'
WHEN `salário` <= 1045 THEN '2º Quintil PNAD'
WHEN `salário` <= 1500 THEN '3º Quintil PNAD'
WHEN `salário` <= 2700 THEN '4º Quintil PNAD'
WHEN `salário` > 2700 THEN '5º Quintil PNAD'end """))

In [136]:
df.groupby('faixapnad').count().toPandas().sort_values('faixapnad')

,faixapnad,count
1,1º Quintil PNAD,605731
0,2º Quintil PNAD,3038235
3,3º Quintil PNAD,15436386
4,4º Quintil PNAD,7842081
2,5º Quintil PNAD,2290694


In [16]:
df.groupby('competência').count().toPandas().sort_values('competência')

,competência,count
9,202001,2890763
11,202002,2985661
7,202003,3172722
6,202004,2251887
4,202005,1867253
0,202006,1914231
3,202007,2159751
10,202008,2335252
1,202009,2544644
8,202010,2789381


In [159]:
df = df.sampleBy('faixapnad',
            fractions= {
                '1º Quintil PNAD':50000/605731,
                '2º Quintil PNAD':50000/3038235,
                '3º Quintil PNAD':50000/15436386,
                '4º Quintil PNAD':50000/7842081,
                '5º Quintil PNAD':50000/2290694
            })

In [160]:
df.createOrReplaceTempView('t1')
df.write.mode('overwrite').format('parquet').save('drive/MyDrive/publicdataml/sample_data_v0/')


In [31]:
spark.sql("""
SELECT `competência`
  ,AVG(`salário`)
  ,std(`salário`)
  ,min(`salário`)
  ,max(`salário`)
  ,percentile_approx(`salário`, 0.25)
  ,percentile_approx(`salário`, 0.5)
  ,percentile_approx(`salário`, 0.75)
  
FROM t1
GROUP BY `competência`
order by 1
""").toPandas()

,competência,avg(salário),stddev_samp(CAST(salário AS DOUBLE)),min(salário),max(salário),"percentile_approx(salário, CAST(0.25 AS DOUBLE), 10000)","percentile_approx(salário, CAST(0.5 AS DOUBLE), 10000)","percentile_approx(salário, CAST(0.75 AS DOUBLE), 10000)"
0,202001,4847.250950,1.574581e+06,0.0,1.800000e+09,1106.000000,1317.729980,1633.900024
1,202002,4297.728758,2.136358e+06,0.0,3.509307e+09,1100.469971,1313.520020,1623.000000
2,202003,8519.115632,3.520114e+06,0.0,3.285988e+09,1114.500000,1320.000000,1622.400024
3,202004,3135.514475,1.491467e+05,0.0,7.579440e+07,1140.599976,1352.000000,1696.839966
4,202005,3190.040084,1.771819e+05,0.0,9.196000e+07,1139.510010,1356.150024,1730.930054
5,202006,3409.318854,8.363704e+05,0.0,1.146780e+09,1137.229980,1356.599976,1720.349976
6,202007,5628.689381,4.313289e+05,0.0,1.671252e+08,1128.380005,1345.119995,1691.829956
7,202008,5990.227498,1.919283e+06,0.0,2.869022e+09,1125.000000,1339.520020,1674.000000
8,202009,4476.895081,7.062848e+05,0.0,1.029000e+09,1123.530029,1333.900024,1650.000000
9,202010,4741.179946,5.051624e+05,0.0,4.531712e+08,1131.520020,1338.479980,1638.000000


In [161]:
import pandas as pd 
df = spark.read.format('parquet').load('drive/MyDrive/publicdataml/sample_data_v0/*')

In [162]:
dicts= {}
dicts.update({'região' :         pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=1) })
dicts.update({'uf' :             pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=2) })
dicts.update({'município' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=3) })
dicts.update({'seção' :     pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=4) })
dicts.update({'subclasse' : pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=5) })
dicts.update({'categoria' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=7) })
dicts.update({'cbo2002ocupação' :            pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=8) })
dicts.update({'graudeinstrução' :      pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=9) })
dicts.update({'sexo' :           pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=10)})
dicts.update({'raçacor' :        pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=17)})
dicts.update({'tamestabjan' :   pd.read_excel('drive/MyDrive/publicdataml/Layout Novo Caged Movimentação.xlsx',sheet_name=19)})

In [163]:
for key, value in dicts.items():
  if key in df.columns: 
    value.columns = value.columns + '_' + key
    value.columns = [key,value.columns[1]]
    spd = spark.createDataFrame(value)#.createOrReplaceTempView(key.encode('ascii', 'ignore').decode('ascii'))
    df = df.withColumn(key,F.col(key).cast('bigint')).join(spd,on=key,how='left')
  else: 
    pass
  

In [164]:
pop_brasil = pd.read_excel('drive/MyDrive/publicdataml/População Brasil.xls',sheet_name=2,header=1,dtype='object')
pop_brasil = pop_brasil.loc[:,['COD. UF','COD. MUNIC','POPULAÇÃO ESTIMADA']]

In [165]:
pop_brasil = pop_brasil.dropna()

In [166]:
pop_brasil.loc[:,'município'] = pop_brasil.loc[:,['COD. UF','COD. MUNIC']].apply(lambda x: str(x[0])+str(x[1]),axis=1).str[:6]
pop_brasil.loc[:,'pop_estimada'] = pop_brasil['POPULAÇÃO ESTIMADA']
pop_brasil.loc[pop_brasil.pop_estimada.str.contains("(|)",na=False),'pop_estimada'] = pop_brasil.loc[pop_brasil.pop_estimada.str.contains("(|)",na=False),'pop_estimada'].str.split('(',expand=True)[0]
pop_brasil.loc[:,'pop_estimada'] = pop_brasil.loc[:,'pop_estimada'].astype('int')

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [167]:
pop_brasil = pop_brasil.groupby('município')['pop_estimada'].sum().reset_index()
pop_brasil.columns= ['município','pop_estimada']

In [168]:
spd = spark.createDataFrame(pop_brasil)#.createOrReplaceTempView(key.encode('ascii', 'ignore').decode('ascii'))
df = df.join(spd,on='município',how='left')


In [169]:
df.write.mode('overwrite').format('parquet').save('drive/MyDrive/publicdataml/sample_data_v1/')

In [170]:
df = spark.read.format('parquet').load('drive/MyDrive/publicdataml/sample_data_v1/*')
df.count()

251090

In [171]:
df.limit(10).toPandas()

,município,tamestabjan,raçacor,sexo,graudeinstrução,cbo2002ocupação,categoria,subclasse,seção,uf,região,competência,saldomovimentação,idade,horascontratuais,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,indicadoraprendiz,fonte,faixapnad,Descrição_região,Descrição_uf,Descrição_município,Descrição_seção,Descrição_subclasse,Descrição_categoria,Descrição_cbo2002ocupação,Descrição_graudeinstrução,Descrição_sexo,Descrição_raçacor,Descrição_tamestabjan,pop_estimada
0,110009,6,6,1,7,784105,999,5320202,None,11,1,202004,-1,19,44,0,1,31,0,0,0,1185.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Serviços de Entrega Rápida,Não Identificado,"Embalador, a Mao",Médio Completo,Homem,Não informada,De 50 a 99 vínculos,32695
1,110009,6,3,1,7,784105,999,5320202,None,11,1,202010,1,23,44,0,1,20,0,0,0,1185.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Serviços de Entrega Rápida,Não Identificado,"Embalador, a Mao",Médio Completo,Homem,Parda,De 50 a 99 vínculos,32695
2,110009,6,3,3,7,513505,999,5320202,None,11,1,202009,1,38,44,0,1,20,0,0,0,1185.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Serviços de Entrega Rápida,Não Identificado,Auxiliar nos Serviços de Alimentação,Médio Completo,Mulher,Parda,De 50 a 99 vínculos,32695
3,110009,9,1,3,9,411010,101,1012101,None,11,1,202007,-1,48,44,0,1,31,0,0,0,4500.0,0,1,5º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Assistente Administrativo,Superior Completo,Mulher,Branca,De 500 a 999 vínculos,32695
4,110009,9,3,3,7,411005,101,1012101,None,11,1,202001,1,20,44,0,1,20,0,0,0,1164.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...","Auxiliar de Escritorio, em Geral",Médio Completo,Mulher,Parda,De 500 a 999 vínculos,32695
5,110009,9,3,3,4,848505,101,1012101,None,11,1,202011,-1,44,44,0,1,40,0,0,0,1104.0,0,1,3º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Abate de Aves,"Empregado - Geral, inclusive o empregado públi...",Abatedor,6ª a 9ª Fundamental,Mulher,Parda,De 500 a 999 vínculos,32695
6,110009,1,6,1,5,772115,101,4744002,None,11,1,202007,1,53,44,0,1,20,0,0,0,2403.5,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Comércio Varejista de Madeira e Artefatos,"Empregado - Geral, inclusive o empregado públi...",Secador de Madeira,Fundamental Completo,Homem,Não informada,Nenhum vínculo,32695
7,110009,1,1,1,7,411005,999,4744001,None,11,1,202010,1,53,44,0,1,20,0,0,0,2000.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Comércio Varejista de Ferragens e Ferramentas,Não Identificado,"Auxiliar de Escritorio, em Geral",Médio Completo,Homem,Branca,Nenhum vínculo,32695
8,110009,1,1,1,6,717020,101,4120400,None,11,1,202002,1,31,44,0,1,20,0,0,0,1663.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Construção de Edifícios,"Empregado - Geral, inclusive o empregado públi...",Servente de Obras,Médio Incompleto,Homem,Branca,Nenhum vínculo,32695
9,110009,1,3,1,6,717020,101,4120400,None,11,1,202010,1,16,44,0,1,20,0,0,0,1663.0,0,1,4º Quintil PNAD,Norte,Rondônia,Ro-Espigao D Oeste,None,Construção de Edifícios,"Empregado - Geral, inclusive o empregado públi...",Servente de Obras,Médio Incompleto,Homem,Parda,Nenhum vínculo,32695
